# Facial Recognition Software

In [22]:
#1. Generate dataset 
#2. Train the classifier and save it 
#3. Detect the face and name it if its already stored in our dataset

# Generate dataset

In [23]:
import cv2
import os
import glob
import numpy as np
from PIL import Image

In [24]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y + h, x:x + w]
        return cropped_face

    cap = cv2.VideoCapture(0)
    directory = "data"
    if not os.path.exists(directory):  # Check if directory exists
        os.makedirs(directory)  # Create directory if it doesn't exist

    # Find the highest user ID in the filenames of the existing images
    files = glob.glob(f"{directory}/user_*.jpg")
    if files:
        user_id = max(int(file.split('_')[1].split('.')[0]) for file in files)
    else:
        user_id = 0

    user_id += 1  # Increment the user ID for the new images

    images_generated = 0  # New variable to count the number of images generated

    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            images_generated += 1  # Increment the count each time an image is generated
            face = cv2.resize(face_cropped(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = os.path.join(directory, f"user_{user_id}_{images_generated}.jpg")
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(images_generated), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow("Cropped face", face)
            if cv2.waitKey(1) == 13 or images_generated == 50:  # Check if 50 images have been generated
                break
    cap.release()
    cv2.destroyAllWindows()
    print(f"Collecting samples for user {user_id} is completed..............")

#generate_dataset()

# Train the classifier and save it

In [25]:
import os 
import cv2
import numpy as np
from PIL import Image

In [8]:
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')  # Convert image to grayscale
        imageNp = np.array(img, 'uint8')        
        
        # Extract ID from filename
        id = int(os.path.split(image)[1].split('_')[1].split('.')[0])

        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids, dtype=np.int32)
    
    # Train the classifier and save
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")

# Call your function with the data_dir
train_classifier("data")


# Detect the face and name it if it is already stored in our dataset

In [1]:
import cv2
import numpy as np
from PIL import Image
import os

In [2]:
def draw_boundray(img,classifier,scaleFactor,minNeighbors,color,text,clf):
    gray_image = cv2.cvtcolor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image,scaleFactor,minNeighbors)
    
    coords = []
    
    for(x,y,w,h) in features:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
        id,pred = clf.predict(gray_image[y:y+h,x:x+w])
        confidence = int(100*(1-pred/300))
        
        if confidence>77:
            if id==1:
                cv2.putText(img,"Serm",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
            if id==2:
                cv2.putText(img,"Raph",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
        else:
            cv2.putText(img,"UNKNOWN",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255),1,cv2.LINE_AA)
            
        coords=[x,y,w,h]
    return coords

def recognize(img,clf,faceCascade):
    coords = draw_boundray(img,faceCascade,1.1,10,(255,255,255),"Face",clf)
    return img 

faceCascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

video_capture = cv2.VideoCapture(0)

while True:
    ret,img = video_capture.read()
    img = recognize(img,clf,faceCascade)
    cv2.imshow("Face Detection",img)
    
    if cv2.waitkey(1)==13:
        break
    
video_capture.release()
cv2.destroyAllWindows()

AttributeError: module 'cv2' has no attribute 'cvtcolor'